In [87]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from tqdm import tqdm

DEVICE_MAPPING = {
    "gpu": torch.cuda,
    "mps": torch.backends.mps,
}

def get_torch_device(device_mapping: dict) -> torch.device:
    """gets the best performant device (CPU or MPS)"""
    for device_name, device in device_mapping.items():
        if device.is_available():
            return torch.device(device_name)

    return torch.device("cpu")


# device = get_torch_device(DEVICE_MAPPING)
device = torch.device("cpu")

print(f"Using device: {device}")

dataset = Planetoid(root='/tmp/Cora', name='Cora')
dataset

Using device: cpu


Cora()

In [88]:
type(dataset)

torch_geometric.datasets.planetoid.Planetoid

In [89]:

class GCN(torch.nn.Module):
    def __init__(self, num_features: int, hidden_size: int, num_classes: int) -> None:
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_size)
        self.hidden = GCNConv(hidden_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=.5, training=self.training)
        # print(f'Input shape: {x.shape}')

        # x = self.hidden(x, edge_index)
        # x = F.relu(x)
        # x = F.dropout(x, p=.75, training=self.training)
        # print(f'Hidden shape: {x.shape}')

        x = self.conv2(x, edge_index)
        # print(f'Output shape: {x.shape}')
        return F.log_softmax(x, dim=1)


In [90]:
model = GCN(num_features=dataset.num_node_features, hidden_size=64, num_classes=dataset.num_classes).to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

model.train()
for epoch in tqdm(range(200)):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    test_loss = F.nll_loss(out[data.test_mask], data.y[data.test_mask])
    if epoch % 10 == 0:  # print every 10th epoch
        print(f'Epoch: {epoch}, Train loss: {loss.item()}, Test loss: {test_loss.item()}')
    loss.backward()
    optimizer.step()

 20%|██        | 41/200 [00:00<00:00, 209.21it/s]

Epoch: 0, Train loss: 1.9512697458267212, Test loss: 1.9435139894485474
Epoch: 10, Train loss: 1.6566219329833984, Test loss: 1.7884819507598877
Epoch: 20, Train loss: 1.3158575296401978, Test loss: 1.5770905017852783
Epoch: 30, Train loss: 0.943773090839386, Test loss: 1.3503844738006592
Epoch: 40, Train loss: 0.6710296273231506, Test loss: 1.1711639165878296


 43%|████▎     | 86/200 [00:00<00:00, 210.91it/s]

Epoch: 50, Train loss: 0.4513654410839081, Test loss: 1.0223459005355835
Epoch: 60, Train loss: 0.33539119362831116, Test loss: 0.919208288192749
Epoch: 70, Train loss: 0.23925192654132843, Test loss: 0.8552830815315247
Epoch: 80, Train loss: 0.19442668557167053, Test loss: 0.8092189431190491
Epoch: 90, Train loss: 0.14211392402648926, Test loss: 0.7684388756752014


 66%|██████▋   | 133/200 [00:00<00:00, 223.96it/s]

Epoch: 100, Train loss: 0.1280958652496338, Test loss: 0.7645316123962402
Epoch: 110, Train loss: 0.09636207669973373, Test loss: 0.7286208271980286
Epoch: 120, Train loss: 0.09051734209060669, Test loss: 0.7305976152420044
Epoch: 130, Train loss: 0.07303255051374435, Test loss: 0.7083525061607361
Epoch: 140, Train loss: 0.0733623132109642, Test loss: 0.7094988822937012


 90%|█████████ | 181/200 [00:00<00:00, 227.95it/s]

Epoch: 150, Train loss: 0.059924207627773285, Test loss: 0.6920695304870605
Epoch: 160, Train loss: 0.059165921062231064, Test loss: 0.6927162408828735
Epoch: 170, Train loss: 0.05422285199165344, Test loss: 0.697075605392456
Epoch: 180, Train loss: 0.049243487417697906, Test loss: 0.6923565864562988
Epoch: 190, Train loss: 0.04270246997475624, Test loss: 0.6847146153450012


100%|██████████| 200/200 [00:00<00:00, 220.68it/s]


In [91]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7940


In [78]:
data.train_mask

tensor([ True,  True,  True,  ..., False, False, False], device='mps:0')

In [92]:
pred

tensor([3, 4, 4,  ..., 1, 3, 3])

In [93]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(data.y[data.test_mask].cpu().numpy(), pred[data.test_mask].cpu().numpy())
cm

array([[101,   4,   4,   6,   5,   2,   8],
       [  6,  80,   3,   2,   0,   0,   0],
       [  3,   7, 131,   3,   0,   0,   0],
       [ 25,  12,  15, 226,  30,   8,   3],
       [  7,   2,   2,   5, 127,   5,   1],
       [ 11,   4,   4,   1,   0,  75,   8],
       [  5,   2,   0,   1,   0,   2,  54]])